In [1]:
!pip install PyPDF2 langchain faiss-cpu -q > /dev/null 2>&1

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import os
import pickle
import json
import numpy as np
import PyPDF2
import faiss
import re

# DEFINICIÓN DE MÉTODOS NECESARIOS

In [3]:
from huggingface_hub import snapshot_download

def load_from_huggingface(repo_name, cache_dir=".rag_cache", repo_type="dataset"):
    """Carga la base de datos RAG desde Hugging Face Hub"""
    try:
        # Descargar todo el repositorio
        downloaded_path = snapshot_download(
            repo_id=repo_name,
            repo_type=repo_type,
            cache_dir=cache_dir,
            allow_patterns=["*.pkl", "*.npy", "*.bin", "*.json"]
        )
        # Cargar chunks
        with open(f"{downloaded_path}/chunks.pkl", "rb") as f:
            chunks = pickle.load(f)

        # Cargar embeddings
        embeddings = np.load(f"{downloaded_path}/embeddings.npy")

        # Cargar índice FAISS
        index = faiss.read_index(f"{downloaded_path}/faiss_index.bin")

        # Cargar metadata
        with open(f"{downloaded_path}/metadata.json", "r", encoding="utf-8") as f:
            metadata = json.load(f)

        # Cargar modelo de embeddings
        model = SentenceTransformer(metadata["model_name"], trust_remote_code=True)

        print(f"✅ Base de datos RAG cargada exitosamente desde: {repo_name}")
        return chunks, embeddings, index, model, metadata

    except Exception as e:
        print(f"❌ Error cargando desde Hugging Face: {e}")
        raise


In [4]:
#Cargar base vectorial
chunks, embeddings, index, model, metadata = load_from_huggingface("aatituanav/rag_LOES")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

chunks.pkl:   0%|          | 0.00/246k [00:00<?, ?B/s]

faiss_index.bin:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

embeddings.npy:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

✅ Base de datos RAG cargada exitosamente desde: aatituanav/rag_LOES


In [5]:
# Búsqueda semántica
def semantic_search(query, model, index, chunks, k=5):
    """
    Realiza una búsqueda semántica para encontrar los fragmentos más relevantes a una consulta.

    Parámetros:
        query: Texto de consulta para la búsqueda
        model: Modelo de embeddings para codificar la consulta
        index: Índice FAISS con los embeddings indexados
        chunks: Lista de fragmentos de texto originales
        k: Número de resultados a retornar (por defecto: 5)

    Retorna:
        list: Lista de los k fragmentos más relevantes a la consulta
    """
    # Generar embedding para la consulta
    query_embedding = model.encode([query], task="retrieval.query")
    # Buscar
    distances, indices = index.search(query_embedding.astype('float32'), k)
    relevant_chunks = [chunks[i] for i in indices[0]]

    return relevant_chunks

In [7]:
#Probar alguna busqueda del LOES
query = "Cuales son las obligaciones del consejo de regentes?"
relevant_chunks2 = semantic_search(query, model, index, chunks)
print(relevant_chunks2)

['Art. 47.2.-Atribuciones del Consejo de Regentes.-Serán deberes y atribuciones del Consejo de\nRegentes:\na) Rendir cuentas e informar al Órgano Colegiado Superior de las actividades relativas al\ncumplimiento de sus funciones, según lo establecido en los estatutos de las instituciones de\neducación superior a la cual pertenecen, o cuando éste lo requiera.\nb) Aprobar la planificación estratégica institucional en el marco de las disposiciones de la\nConstitución y la ley, promoviendo la articulación con el desarrollo nacional.\nc) Proponer o elegir, de ser el caso, y conforme los mecanismos previstos en esta Ley, el Rector o\nRectora, Vicerrector o Vicerrectora, respetando el principio de alternabilidad.\nd) Solicitar la remoción del Rector o Rectora, Vicerrector o Vicerrectora, respetando el debido\nproceso y conforme a las causales y al procedimiento determinado en esta Ley y su reglamento.\ne) Las demás que establezca el estatuto de la institución de educación superior, conforme a 

In [8]:
#Configurar API de DeepSeek con sintaxis actualizada
from openai import OpenAI
import pandas as pd
import json

client = OpenAI(
    api_key="sk-d2857e84df9e4e30a056b3cab70254f9",
    base_url="https://api.deepseek.com/v1"
)

def callPromptLLM(prompt, role):
    """
    Realiza una llamada al modelo de lenguaje DeepSeek con un prompt y rol específico.
    Parámetros:
        prompt: Texto de entrada para el modelo
        role: Rol del sistema que define el contexto de la interacción
    Retorna:
        str: Respuesta generada por el modelo o None en caso de error
    """
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": role},
                {"role": "user", "content": prompt}
            ],
            max_tokens=5000,
            temperature=0.3,
            top_p=0.9,
            frequency_penalty=0.3,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error procesando")
        print(e)
        return None

In [10]:
def generate_context(keywords):
    # Transformar una lista de palabras clave en consultas de búsqueda
    role = "Eres un experto en análisis semántico y búsqueda de información legal-académica, especializado en la Ley Orgánica de Educación Superior (LOES) del Ecuador y su marco normativo relacionado."
    prompt = f'''
TAREA: Tu tarea es analizar las palabras clave proporcionadas y generar una "expansión de consulta" No te limites a buscar sinónimos. Debes inferir la intención normativa detrás de las palabras y generar una lista de frases y términos de búsqueda precisos que probablemente se encuentren en el texto de la LOES.
Los términos que generes deben optimizar la recuperación de artículos, reglamentos y procedimientos relevantes que luego servirán como **considerandos legales** en la elaboración de un borrador.
CONTEXTO: Estoy utilizando un sistema de Búsqueda y Generación Aumentada (RAG) que tiene acceso exclusivo a una base de datos vectorial con el texto completo y actualizado de la "Ley Orgánica de Educación Superior del Ecuador". El objetivo final es identificar los artículos exactos de esta ley que justifiquen y fundamenten la creación de un nuevo borrador de reglamento o procedimiento basado en un conjunto de palabras clave.

INSTRUCIONES:
1. **Contextualización legal:**
   - Relaciona las palabras clave con conceptos jurídicos, estructuras administrativas o procesos académicos definidos en la LOES (ej: "matrícula" → "proceso de matrícula", "derechos académicos", "requisitos de inscripción").
2. **Sinónimos y jerarquía normativa:**
   - Incluye sinónimos legales (ej: "estudiante" → "educando", "sujeto de derecho educativo"), términos técnicos (ej: "régimen académico") y referencias a secciones de la ley (ej: "Capítulo III, Título IV").
4. **Precisión para RAG:**
   - Prioriza términos que probablemente aparezcan textualmente en la LOES (ej: "fraude académico" → "faltas disciplinarias", "procedimiento sancionatorio").

### Ejemplo:
**Palabras clave de entrada:** "matrícula", "derechos estudiantes"
**Términos generados:**
1. Proceso de matrícula y inscripción
2. Requisitos de matrícula ordinaria y extraordinaria
3. Derechos y obligaciones de los estudiantes
4. Régimen académico estudiantil
5. Garantías procedimentales en matrícula
7. Reglamento de registro académico
8. Cancelación de matrícula
9. Derecho a la participación estudiantil
10. Reclamos por negativa de matrícula

FORMATO DE SALIDA: Presenta 5 terminos de busqueda en diferentes lineas. No des ninguna explicación adicional.

PALABRAS CLAVE: {",".join(keywords)}
    '''
    numberRelevantChunks_k = 3
    potentialQueriesString = callPromptLLM(prompt, role)
    queries = potentialQueriesString.splitlines()
    print("Términos clave: \n"+potentialQueriesString)
    all_relevant_chunks = []
    for query in queries:
        relevant_chunks = semantic_search(query, model, index, chunks, k=numberRelevantChunks_k)
        all_relevant_chunks.extend(relevant_chunks)
    # Eliminar duplicados manteniendo el orden
    seen = set()
    unique_chunks = []
    for chunk in all_relevant_chunks:
        if chunk not in seen:
            seen.add(chunk)
            unique_chunks.append(chunk)
    # Limitar a los 15 chunks más relevantes
    # Configurar esto dependiendo de la cantidad de chunks relevantes encontrados (k)
    #y de la longitud de "queries"
    maxChunks = max(15, len(queries)*numberRelevantChunks_k)
    context = "\n\n".join(unique_chunks[:maxChunks])
    return context

In [11]:
def generate_draft_document(keywords, context):
    #Genera un borrador de documento basado en el contexto

    role = "Eres un asistente experto en redacción de documentos oficiales para instituciones de educación superior en Ecuador, con amplio conocimiento de la Ley Orgánica de Educación Superior (LOES) y normativa interna institucional."
    prompt = f"""

TAREA: Generar un borrador de resolución completo y formal para el Instituto Superior Tecnológico Yaruquí. Para ello, recibirás dos elementos clave: PALABRAS CLAVE que resumen la decisión a tomar, y un conjunto de LEYES Y NORMATIVAS que servirán de base para la argumentación. El borrador debe reflejar fielmente la intención de las palabras clave y estar fundamentado en las normas proporcionadas.

INSTRUCCIONES:
*Analizar las PALABRAS CLAVE: Lee atentamente las PALABRAS CLAVE para identificar la intención principal y los elementos centrales de la resolución.
*Redactar la sección 'RESUELVE': Basándote en el análisis de las PALABRAS CLAVE, redacta uno o varios artículos claros, concisos y en lenguaje formal y jurídico que materialicen la decisión. Cada artículo debe corresponder a una acción específica. Por ejemplo, si las palabras clave son 'aprobacion, adquisicion equipos, designacion a Andres Tituana', deberás crear un artículo para la aprobación de la adquisición y otro para la designación de la persona encargada.
*Analizar las LEYES Y NORMATIVAS: Revisa el contexto legal proporcionado. Identifica y extrae los artículos, estatutos o principios que justifican y dan fundamento a las decisiones que se redactan en la sección 'RESUELVE'.
*Redactar la sección 'CONSIDERANDO': Utiliza las normativas relevantes que identificaste para elaborar los 'considerandos'. Cada considerando debe ser un párrafo que explique el marco legal o la razón de ser de la resolución, citando o parafraseando las leyes proporcionadas. Debes conectar lógicamente el marco normativo con la decisión que se tomará.
*Ensamblar la Resolución Final: Integra las secciones CONSIDERANDO y RESUELVE en la plantilla de formato estricto que se proporciona a continuación. No alteres la estructura fija del documento. Completa los campos variables como el número de resolución y la fecha con placeholders (corchetes) si no se proporcionan.
*Formato estricto (No modificar esta estructura)
  ```
  RESOLUCIÓN: [NUMERO_RESOLUCION]

  EL ÓRGANO COLEGIADO SUPERIOR DEL INSTITUTO SUPERIOR TECNOLÓGICO YARUQUÍ

  CONSIDERANDO:
  (Aquí elabora los considerandos basándote en las normas proporcionadas que más reflejen la resolución que se va a crear, para ello analiza cada contexto que se te va a proporcionar sobre las normas)

  RESUELVE:
  (Aquí colocas la resolución, redactada en uno o más artículos, basada en las palabras clave)

  DISPOSICIONES FINALES:

  PRIMERA.- La presente resolución entrará en vigencia a partir de la fecha de suscripción de la misma.
  SEGUNDA.- Comuníquese y cúmplase.

  Dado en la ciudad de Quito, a los [FECHA], en la [NUMEROSESSION] sesión [TIPOSESSION] del Órgano Colegiado Superior del Instituto Superior Tecnológico Yaruquí.

  Mgs. Isabel Rodriguez
  RECTORA

  Lcda. Melissa Aguilar
  SECRETARIA
  ```
PALABRAS CLAVE ###{keywords}###
LEYES Y NORMATIVAS ###{context}###
    """
    template = callPromptLLM(prompt, role)
    return template

# UTILIZACIÓN
Esperar un momento en lo que se genera el borrador ...


In [14]:
# Procesar consulta del usuario
keywords = ["Andres Tituana", "consejo directivo", "dimision"]

# Generar contexto
context = generate_context(keywords)

# Generar borrador
borrador = generate_draft_document(keywords, context)

print(borrador)

Términos clave: 
Dimisión del consejo directivo
Procedimiento de renuncia de autoridades institucionales
Causales de cese de miembros del consejo directivo
Reemplazo y sucesión en órganos de gobierno universitario
Atribuciones del consejo directivo en casos de dimisión
```  
RESOLUCIÓN: [NUMERO_RESOLUCION]

EL ÓRGANO COLEGIADO SUPERIOR DEL INSTITUTO SUPERIOR TECNOLÓGICO YARUQUÍ

CONSIDERANDO:
Que, de conformidad con lo establecido en el Art. 64.1 de la Ley Orgánica de Educación Superior, las máximas autoridades de las Instituciones de Educación Superior podrán ser removidas por el Consejo de Educación Superior, previa solicitud de al menos las dos terceras partes del órgano colegiado superior, en los casos específicamente determinados;

Que, según lo dispuesto en el Art. 52 de la LOES, el estatuto de toda institución de educación superior contemplará la subrogación o reemplazo del rector o rectora, vicerrectores o vicerrectoras y autoridades académicas en caso de ausencia temporal o de